In [1]:
from docx import Document
import os
import re
import csv
import sys

In [2]:
'''
Author: Eden

Read stopwords from files and return a stopwords set.

-filename = file name

return the set of stopwords
'''
def read_stop_words(filename):
    vector = []
    with open(filename, 'r') as f:
        input = f.read().splitlines()

    [vector.append(input[i]) for i in range(len(input))]   
    return set(vector)

In [3]:
'''
Author: Zoe

Remove the stopwords such as articles, prep and so on from keywords

-keywords = search word

return the string without stopwords
'''

def remove_stopwords(keywords, stop_set):
    word_list = re.split(r'\s',keywords)
    filtered_words = [word for word in word_list if word not in stop_set]
    words = " ".join(filtered_words)
    return words

### indent(para) Example
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Change of control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [4]:
'''
Author: Yufei

Get the indent space of the paragraph, used in find_patterns and match function

-para = paragraph

return a num means the indent space or null if no indent
'''
def indent(para):
    if para.paragraph_format.first_line_indent:
        return para.paragraph_format.first_line_indent.pt
    else:
        return None

### bullet_list(para) Example
Change of control means that 

&nbsp;&nbsp;&nbsp;&nbsp;(a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

&nbsp;&nbsp;&nbsp;&nbsp;(b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

#### The difference between bullet_list and isListPara is that the bullet in the bullet_list is manually input rather than the Microsoft Word style


In [5]:
'''
Author: Eden

Check if the paragraph is bullet ordered list, used in find_patterns and match function

-para = paragraph

return the regex pattern of this paragraph or None
'''
def bullet_list(para):
    pattern = '[·]' # match the bullet point e.g.   · This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])   
    if flag:
        return pattern
    
    pattern = u'[\u2022]' # match the bullet point e.g.   • This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])      
    if flag:
        return pattern
    
    pattern = '[A-Z]+\)' # match the upper case numbering point e.g.   A) This is the content (A) This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])      
    if flag:
        return pattern
    
    pattern = '[a-z]+\)' # match the lower case numbering point e.g.   a) This is the content (a) This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\)' # match the digital numbering point e.g.   1) This is the content  (1) This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\.[^\d]+' # match the digital numbering point e.g.   1. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '[a-z]+\.' # match the lower case numbering point e.g.   a. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '[A-Z]+\.' # match the upper case numbering point e.g.   A. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    
    pattern = '\d+\.\d+[^\d]+' # match the digital numbering point e.g.   1.1. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\.\d+\.\d+[^\d]+' # match the upper case numbering point e.g.   1.1.1. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '\d+\.\d+\.\d+\.\d+[^\d]+' # match the upper case numbering point e.g.   1.1.1.1. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    pattern = '[\(]?([vx]|i{1,3}|i[vx]|[vx]i{1,3})(\.|\))' # match the roman numerals point 
    # e.g. i) This is the content    (i) This is the content    i. This is the content
    flag = re.match(pattern, para.text.lstrip()[:10])
    if flag:
        return pattern
    
    return None

In [6]:
# '''
# Testing Function
# '''

# # Input folder path
# directory_path = os.getcwd() + '/input/change_of_control+change_in_control' 
# fileName = 'test.docx'
# document = Document('/Users/yangyi/Desktop' + "/" +fileName)
#     #
# for p in document.paragraphs:
# #     print p.text
#     pattern = bullet_list(p)
#     if pattern:
#         print pattern
#         print p.text

### isListPara(para) Example
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [7]:
'''
Author: Alan

Check if the paragraph is list paragraph rather than normal paragraph, used in find_patterns and match function

-para = paragraph

return True/False
'''
def isListPara(para):
    paraFormat = para.style.name
    if paraFormat == 'List Paragraph':
        return True
    return False

### isNormal(para) Example
any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 

In [8]:
'''
Author: Alan

Check if the paragraph is Normal rather than list paragraph，used in match function

-para = paragraph

return True/False
'''
def isNormal(para):
    paraFormat = para.style.name
    if paraFormat == 'Normal':
        return True
    return False

### isNormalFollwingListPara(paras, curIndex) Example
Change of control means that 
1. any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or 
2. a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [9]:
'''
Author: Alan

Check if the paragraph followed by list paragraph, used in find_patterns function

-paras = all the paragraph
-curIndex = the index of current paragraph

return True/False
'''
def isNormalFollowingListPara(paras, curIndex):
    targetFormat = paras[curIndex].style.name
    if curIndex == len(paras) - 1:
        return False
    nextFormat = paras[curIndex+1].style.name
    if nextFormat == 'List Paragraph' and targetFormat == 'Normal':
        return True
    return False

In [10]:
def isBullet1FollowingBullet2(paras, index):
    startPara = paras[index]
    bullet1 = bullet_list(startPara)
    
    nextIndex = index + 1
    if nextIndex == len(paras):
        bullet2 = None
    else:
        nextPara = paras[nextIndex]
        bullet2 = bullet_list(nextPara)
        
    #print ('Bullet1, ', bullet1)
    #print ('Bullet2, ', bullet2)
    
    return (bullet1 != None or bullet2 != None)

### bold(para) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

**Code** shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [11]:
'''
Author: Zheng

Check if the bold font in the paragraph, used in match function

-para = paragraph

return True/False
'''

def bold(para):
    paras_toPrint = []
    i=0
    for run in para.runs:
#         count the number of run in the start of the paragraph
        if i>3:
            break
        i=i+1
        if run.bold:
            paras_toPrint.append(run.text) 
#     if there is no bold word, return false, otherwise return true
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_bold(para, keywords) Example
**Change of control** means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [12]:
'''
Author: Zheng

Check if the bold font and keyword in the paragraph, used in find_patterns function

-para = paragraph
-keywords = search words

return True/False
'''

def is_bold(para, keywords):
    paras_toPrint = []
    i=0
#   check whether the paragraph contains the keyword
    if keywords.lower() in para.text.lower(): 
        for run in para.runs:
#           check whether the bold word is in the start of the paragraph.
            if i>1:  
                break
            i=i+1
            if run.bold:
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_italic(para, keywords) Example
_Change of control_ means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [13]:
'''
Author: Zheng

Check if the italic font and keywords in the paragraph, used in match function

-para = paragraph

return True/False
'''
def is_italic(para, keywords):
    paras_toPrint = []
    i=0
    if keywords.lower() in para.text.lower(): 
        for run in para.runs:
            if i>1:  
                break
            i=i+1
#           check whether the italic word is in the start of the paragraph.
            if run.italic:
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### italic(para) Example
_Code_ shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [14]:
'''
Author: Zheng

Check if the italic font in the paragraph, used in match function

-para = paragraph

return True/False
'''

def italic(para):
    paras_toPrint = []
    i=0
    for run in para.runs:
#       count the number of run in the start of the paragraph
        if i>3:
            break
        i=i+1
        if run.italic:
            paras_toPrint.append(run.text) 
    if  len(paras_toPrint)==0:
        return False
    else:
        return True

### is_underline(para, keywords) Example
Change of control(Underline) means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [15]:
# Zheng
'''
Author: Zheng

Check if the keyword is underlined in the paragraph, used in find_pattern function

-para = paragraph
-keywords = keywords

return True/False
'''

def is_underline(para, keywords):
    paras_toPrint = []
    i=0
#   check whether the keyword in the paragraph
    if keywords.lower() in para.text.lower(): 
        for run in para.runs:
#           the word with underline should appear in the start of the paragraph
            if i>1:  
                break
            i=i+1
            if run.underline:
                paras_toPrint.append(run.text)
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### underline(para) Example

Code(Underline) shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [16]:
'''
Author: Zheng

Check if the underline in the paragraph, used in match function

-para = paragraph

return True/False
'''

def underline(para):
    paras_toPrint = []
    i=0
    for run in para.runs:
#       count the number of run in the start of the paragraph
        if i>3:
            break
        i=i+1
        if run.underline:
            paras_toPrint.append(run.text) 
    if len(paras_toPrint) == 0:
        return False
    else:
        return True

### is_double_quotes(para, keywords) Example:
"Change of control" means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

In [17]:
'''
Author: Zoe

Check if the keyword is double quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_double_quotes(para, keywords, stop_set):
    words = re.split(u'[\"\u201c\u201d]',para.text)
    removed_words = remove_stopwords(keyword, stop_set).split(" ")
    if len(words) == 1:  # no double quotes
        return False
    # check if all keywords are in words[1], the first phrase in double quotes, and words[0] is not regular phrase which means that words[1] is the start of the paragragh
    if all(word in words[1] for word in removed_words) and len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True
    return False

### double_quotes(para) Example:

"Code" shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [18]:
'''
Author: Zoe

Check if the double quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def double_quotes(para):
    """
        Find if paragraph starts with double quotes
        
        return: True or False
    """
    words = re.split(u'[\"\u201c\u201d]',para.text)
    #print words
    if  len(words) == 1:
        return False
    # if words[0] is not regular phrase, we regard  words[1] as the start of the paragragh
    if len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True       
    else:
        return False

### is_single_quotes(para, keywords) Example:
'Change of control' means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [19]:
'''
Author: Zoe

Check if the keyword is single quoted in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_single_quotes(para, keywords, stop_set):
    words = re.split(u'[\"\u2018\u2019]',para.text)
    removed_words = remove_stopwords(keyword, stop_set).split(" ")
    #re.findall(r'[\"\u201C\u201D].*[\"\u201C\u201D]', para.text)
    #print words
#     print '\tIS Double Quotes', words
    if len(words) == 1:    # no single quotes
        return False
        # check if all keywords are in words[1], the first phrase in single quotes, and words[0] is not regular phrase which means that words[1] is the start of the paragragh
    if all(word in words[1] for word in removed_words) and len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True
    return False

### single_quotes(para) Example:

'Code' shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [20]:
'''
Author: Zoe

Check if single quotes in the paragraph, used in match function

-para = paragraph

return True/False
'''
def single_quotes(para):
    words = re.split(u'[\"\u2018\u2019]',para.text)
    if len(words) == 1:
        return False
    # if words[0] is not regular phrase, we regard words[1] as the start of the paragragh
    if len(words[0].rstrip().lstrip()) < 7 and re.match(r'[a-zA-Z ]*[a-zA-Z]+[a-zA-Z ]*',words[0]) == None:
        return True       
    else:
        return False

### is_upper_case(para, keywords) Example:
CHANGE OF CONTROL shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [21]:
'''
Author: Zoe

Check if the keyword is upper case in the paragraph, used in find_patterns function

-para = paragraph

return True/False
'''
def is_upper_case(para, keywords):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]): # and words[0].lower() == keyword.lower():
        return True
    else:
        return False

### upper_case(para) Example:

UPPERCASE shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.


In [22]:
'''
Author: Zoe

Check if upper case word in the paragraph, used in match function

-para = paragraph

return True/False
'''
def upper_case(para):
    words = re.findall(r'([A-Z]+?)\s',para.text)
    if not words:
        return False
    if para.text.startswith(words[0]):
        return True
    else:
        return False

#### Remove_Stopwords Test

In [23]:
'''
Testing Function
'''
directory_path = 'res' 
fileName = 'stop_words.txt'
stop_set = read_stop_words(directory_path + '/' +  fileName)
keyword = "Change of Control"
print remove_stopwords(keyword, stop_set)

Change Control


In [24]:
'''
Author: Eden

Still Testing

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search_2(paras, keywords, stop_set):
    index = []
    i = 0
    # remove stop words and get words list
    target = remove_stopwords(keywords.lower(), stop_set)
    target_words = re.split('[\s]', target)
    
    # iterate all the paragraph and return the index of the paragraph which contains key words
    while (i<len(paras)):
        p = paras[i]
        # use sliding window to get key words
        if contains_sliding_window(p.text.lower(), target_words, 2 * len(keywords)):
            index.append(i)
        i = i+1
    return index

In [25]:
'''
Author: Eden

Still Testing

Find the keywords by sliding windows so that even the order of keywords change, we can still fetch the results

-text = all paragraphs
-keywords = search words
-win_len = the length of sliding window

return True if the keyword is in text within a sliding window by random orders.
'''
def contains_sliding_window(text, target_words, win_len):
    probe = target_words[0] # probe is the first word of target words list
    
    flag = False # flag is used to check whether we find all key words in the given text
    idx = 0
    while (True):
        text = text[idx:] # search the remaining text after last search
        if probe in text:
            idx = text.index(probe) # get the index of probe first appearance in text
            l_pointer = max(0,idx - win_len) # determine the left border of sliding window
            r_pointer = min(len(text),idx + win_len) # determine the right border of sliding window
            window_str = text[l_pointer:r_pointer] # substr the content of sliding window 
            flag = True
            for target_word in target_words:  # check whether every word exist in sliding window, 
                if target_word not in window_str:  # if any word miss in sliding windows string, flag will be false
                    flag = False
            if flag:
                return True
            else:
                idx = idx + 1  # increase index to skip first character and avoid infinite loop
        else:
            return False # if there is not probe in the text any more, return false
    return False


In [26]:
'''
Testing Function
'''
text = 'aaChange of Control means an event or series of events by which:'
if contains_sliding_window(text.lower(), 'change control', 2 * len(keyword)):
    print text + '1'

aaChange of Control means an event or series of events by which:1


### upper_camel_case(para, keywords) Example:
Capitalized Lease Obligations means, with respect to any Person, all outstanding obligations of such Person in respect of Capital Leases, taken at the capitalized amount thereof accounted for as indebtedness in accordance with GAAP.

Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.

Code shall mean the Internal Revenue Code of 1986, as the same may be amended or supplemented from time to time, and any successor statute of similar import, and the rules and regulations thereunder, as from time to time in effect.

In [27]:
'''
Author: Zoe and Zheng

Check if the words following upper camel rule in the paragraph, used in match function

-para = paragraph
-stop_set = stop words

return True/False
'''
# find pattern like Change of Control
def upper_camel_case(para, stop_set):
#   split the paragraph into words
    words_list = re.split(" ", para.text) 
#   remove the stopword
    new_para = remove_stopwords(para.text, stop_set)
    new_para_list = re.split(" ", new_para)
#   find all the words with the first letter uppercase
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_para)
    last_word = ""
#   get the last word which has captical letter
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_para_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
#   grab all the words before the last word that is with the first letter uppercase.
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
#   check whether the keyword is contained in the camel pattern phrase.
    if len(result) > 1:
        return True
    else:
        return False

### is_upper_camel_case() Example:
Change of Control means that (a) any Person or group (within the meaning of Rule13d-5 of the SEC under the Exchange Act) shall become the Beneficial Owner of 20% or more of the Voting Equity Interests of the Borrower, or (b) a majority of the members of the Board of Directors of the Borrower shall cease to be Continuing Members.


In [28]:
'''
Author: Zoe and Zheng

Check if the keywords is following upper camel rule in the paragraph or not, used in find_pattern function

-para = paragraph
-keyword = keyword
-stop_set = stop words

return True/False
'''
def is_upper_camel_case(paragraph, keyword, stop_set):
#   split the paragraph into words
    words_list = re.compile('\w+').findall(paragraph.text)
#   remove the stopword
    new_paragraph = remove_stopwords(paragraph.text, stop_set)
#   find all the words with the first letter uppercase

    new_paragraph_list = re.compile('\w+').findall(new_paragraph)
    new_words_list = re.findall(r'([A-Z][a-z0-9\']+)[^A-Z]?\s', new_paragraph)

    last_word = ""
#   get the last word which has captical letter 
    for i in range(len(new_words_list)):
        if new_words_list[i] == new_paragraph_list[i]:
            last_word = new_words_list[i]
        else:
            break
    result = []
#   grab all the words before the last word that is with the first letter uppercase.
    for j in range(len(words_list)):
        if last_word == words_list[j]:
            result.append(last_word)
            break
        else:
            result.append(words_list[j])
    results = " ".join(result)
#   check whether the keyword is contained in the camel pattern phrase.
    if keyword.lower() in results.lower():
        return True
    else:
        return False

In [29]:
'''
Author: Yufei

Traversing through each of the paragraphs and find the paragraph containing the keywords

-paras = all paragraphs
-keywords = search words

return a list of paragraph index 
'''
def search(paras, keywords):
    index = []
    i = 0
    while (i<len(paras)):
        p = paras[i]
        #print(p.text)
        if keywords.lower() in p.text.lower():
            #print("123")
            index.append(i)
        i = i+1
    
    return index

In [30]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-index = paragraph index
-keyword = search words

return True/False list, 
[bold, italic, underline, double_quotes, single_quotes, upper case, (upper camel case), indent space, list paragraph, normal following list paragraph, bullet list]

'''
def find_patterns(paras, index, keyword, stop_set):
    paragraph = paras[index]
    patterns = []
    patterns.append(is_bold(paragraph, keyword))
    patterns.append(is_italic(paragraph, keyword))
    patterns.append(is_underline(paragraph, keyword))
    patterns.append(is_double_quotes(paragraph, keyword, stop_set))
    patterns.append(is_single_quotes(paragraph, keyword, stop_set))
    patterns.append(is_upper_case(paragraph, keyword))
    patterns.append(is_upper_camel_case(paragraph, keyword, stop_set))
    patterns.append(indent(paragraph))
    patterns.append(isListPara(paragraph))
    patterns.append(isNormalFollowingListPara(paras, index))
    patterns.append(isBullet1FollowingBullet2(paras, index))
    patterns.append(bullet_list(paragraph))
    return patterns

In [31]:
'''
Author: Yufei

Find the pattern of the paragraph containing the keywords by call the is_* function

-paras = all paragraphs
-targetPatterns = the target pattern
-startIndex = paragraph index

return content
'''
def match(paras, targetPatterns, startIndex):
    
    i = startIndex + 1
    match = []
    
    print '\tTarget Patterns: ' + str(parttern_translate(targetPatterns))
    
    n = 0
    while (i<len(paras)):
        
        patterns = []
        p = paras[i]
        
        if p.text == "":
            continue
        

        patterns.append(bold(p))
        patterns.append(italic(p))
        patterns.append(underline(p))
        patterns.append(double_quotes(p))
        patterns.append(single_quotes(p))
        patterns.append(upper_case(p))
        patterns.append(upper_camel_case(p, stop_set))
        patterns.append(indent(p))
        patterns.append(isListPara(p))
        patterns.append(isNormalFollowingListPara(paras, i))
        patterns.append(isBullet1FollowingBullet2(paras, i))
        patterns.append(bullet_list(p))
        
        print '\n\tNext Paragraph:\t{0}\n\n\t\t{1}'.format(str(parttern_translate(patterns)), p.text.encode("utf-8"))
        n = n + 1
        if patterns == targetPatterns:
            print 'Bingo Bingo Bingo Bingo'
            break
        i = i + 1
        # not find same pattern
        if n > 10 or i == len(paras):
            match.append(paras[startIndex])
            i = startIndex
            break
    for x in range(startIndex, i):
        match.append(paras[x])
        
    return match, i-1

In [32]:
'''
Author: Eden

return the list of patterns' name

-pattern = the pattern list with true/false

return list of patterns name
'''
def parttern_translate(pattern):
    # This function is for testing, return the list of patterns' name so that it is easier check the patterns match
    list = []
    if pattern[0]:
        list.append('blod')
    if pattern[1]:
        list.append('italic')
    if pattern[2]:
        list.append('underline')
    if pattern[3]:
        list.append('double_quotes')
    if pattern[4]:
        list.append('single_quotes')
    if pattern[5]:
        list.append('upper_case')
    if pattern[6]:
        list.append('upper_camel_case')
    if pattern[7]:
        list.append(pattern[7])
    if pattern[8]:
        list.append('list_paragraph')
    if pattern[9]:
        list.append('normal_following_list_paragraph')
    if pattern[10]:
        list.append('Bullet1_Following_Bullet2')
    if pattern[11]:
        list.append('bullet_list')
    return list
    

In [33]:
# # Abhay
# '''
# Open the output file in append mode and pull related language from the files in the input folder
# '''

# ########################################################################
# out_filepath = 'Output/'
# out_filename = 'Output_search.csv'
# out_path = out_filepath + out_filename
# ########################################################################


# header = ['File Name', 'Keywords', 'Change of Control'] # Column names in output file 

# with open(out_path, 'wb') as f:
#     wr = csv.writer(f)
#     wr.writerow(header)

# with open(out_path, 'ab') as f:
#     # Input folder path
#     directory_path = 'input' 
    
#     # Parse through each file in the input folder path
#     for fileName in os.listdir(directory_path):
#         print fileName
        
#         # Create a Document object of each of the files
#         #document = Document(directory_path + "/" +fileName)
#         document = Document('input/upper_camel_case+underline+after_list_paragraph.docx')
        
        
#         # A list to add all the paragraphs in the document
#         paras = []
        
#         '''
#         The following for loop is used to add all the paragraphs in the document to the paras list object
#         '''
#         for p in document.paragraphs:        
#             paras.append(p)
            
#         paras_number = len(paras) # Find the number of paragraphs in the document
        
#         paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
#         keyword = "Change of Control"
#         keywordindex = search(paras, keyword)
        
#         print('keywordIndex:', keywordindex)
        
#         results = []
#         result = ""
#         for index in keywordindex:
#             print 'Target Para', paras[index].text
#             target = find_patterns(paras, index, keyword)
            
#             if index == paras_number - 1:
#                 result = paras[index]
#             else:
#                 result = match(paras, target, index)
            
#             results.append(result)

#         wr = csv.writer(f)
#         wr.writerow([fileName, keyword, results])
        
# print("end")
# f.close()

In [34]:

'''
Main Function.
'''

########################################################################
out_filepath = 'test_3/output/'
out_filename = 'Output_search.csv'
out_path = out_filepath + out_filename
########################################################################


header = ['File Name', 'Keywords', 'Content'] # Column names in output file 

with open(out_path, 'wb') as f:
    wr = csv.writer(f)
    wr.writerow(header)
    
with open(out_path, 'ab') as f:
    # Input folder path
    directory_path = 'test_3/input/' 
    
    stop_set = read_stop_words('res/stop_words.txt')    
    
    # Parse through each file in the input folder path
    print 'Keyword: ' + 'Change of Control' + '\n'
    for fileName in os.listdir(directory_path):
        if fileName == '.DS_Store':
            continue
        if fileName.startswith('~$'):
            continue
        print '-------------------------------------------------------------------------'
        print 'File name: ' + fileName + '\n'
        
        # Create a Document object of each of the files
        document = Document(directory_path + "/" +fileName)
        # document = Document('/upper_camel_case+underline+after_list_paragraph.docx')
        
        
        # A list to add all the paragraphs in the document
        paras = []
        
        '''
        The following for loop is used to add all the paragraphs in the document to the paras list object
        '''
        for p in document.paragraphs:        
            paras.append(p)
            
        paras_number = len(paras) # Find the number of paragraphs in the document
        
        paras_toPrint = [] # A list to add all the paragraphs relevant to 'Change of Control' language in this case
        
        keyword = "Change of Control"
#         keywordindex = search(paras, keyword)
        keywordindex = search_2(paras, keyword, stop_set)
        
#         print '\tkeywordIndex: ' + str(keywordindex)
#         print
        
        results = []
        result = ""
        endindex = -1
        for index in keywordindex:
            if index <= endindex:
                continue
            if not keyword.lower() in paras[index].text[:100].lower():
                results.append(paras[index])
                endindex = index
                print '\tTarget Paragraph: [{0}]\n\t\t{1}\n'.format(index, paras[index].text.encode("utf-8"))
                continue
            print '\tTarget Paragraph: [{0}]\n\t\t{1}\n'.format(index, paras[index].text.encode("utf-8"))
            target = find_patterns(paras, index, keyword, stop_set)
            

            if index == paras_number - 1:
                result = paras[index]
            else:
                result, endindex = match(paras, target, index)
            
            results.append(result)

        wr = csv.writer(f)
        # Created a single list of all instances of 'Change of Control'
        # Ignored unrecognized ascii characters that throw an encoding error. Need improvement
        for result in results:
            if hasattr(result, '__iter__'):
                text =  [paras.text.encode('ascii','ignore') for paras in result]
                res = "\n".join(text)
            else:
                res = result.text.encode('ascii','ignore')
            
            wr.writerow([fileName, keyword, res])
        
        print '\n'
print("end")
f.close()

Keyword: Change of Control

-------------------------------------------------------------------------
File name: change_in_control_2_Mylan.docx

	Target Paragraph: [9]
		          1.2 “Cause” means (a) for purposes of a termination of employment (other than during the Change in Control Protection Period): (i) the failure by the Eligible Employee to substantially perform the Eligible Employee’s duties (other than any such failure resulting from the Eligible Employee’s incapacity due to physical or mental illness), (ii) the continued failure by the Eligible Employee to perform his duties at a satisfactory level of performance after written notification from his or her manager or supervisor of such failure and after having been provided with a reasonable opportunity to cure such failure, or (iii) the engaging by the Eligible Employee in conduct which is injurious to the Company, monetarily or otherwise; and (b) for purposes of a termination during the Change in Control Protection Period: 


	Next Paragraph:	['double_quotes', 'upper_camel_case']

		(c) “Cause” means the Company’s reasonable, good faith determination that termination of the Executive’s employment is necessary by reason of: (i) failure to perform the requirements of Executive’s position after multiple warnings and a failure to cure; (ii) Executive’s conviction of or plea of “guilty” or “no contest” to a felony or misdemeanor involving dishonesty, or a felony or misdemeanor not involving dishonesty but resulting in a custodial penalty and/or the creation of material reputational risk for the Company; (iii) fraud or intentional misconduct in the performance of Executive’s duties; (iv) the material breach by the Executive of a Company policy, causing financial or reputational harm; or (v) neglect of responsibilities or insubordination; provided, however, that prior to termination for acts subject to (i), (iv) and/or (v) the Company shall provide the Executive with written notice of the actions underlying such 

KeyboardInterrupt: 

In [ ]:
os.getcwd()